In [1]:
import pandas as pd
import ftfy

In [2]:
# Load the CSV file with low_memory=False and without specifying dtypes initially
df = pd.read_csv('raw_movie_data.csv', low_memory=False)

# Convert relevant columns to numeric, coercing errors into NaN
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
df['Duration'] = pd.to_numeric(df['Duration'], errors='coerce')
df['Budget'] = pd.to_numeric(df['Budget'], errors='coerce')
df['Revenue'] = pd.to_numeric(df['Revenue'], errors='coerce')
df['Ratings'] = pd.to_numeric(df['Ratings'], errors='coerce')
df['Vote Count'] = pd.to_numeric(df['Vote Count'], errors='coerce')
df['Popularity'] = pd.to_numeric(df['Popularity'], errors='coerce')
df['Movie ID'] = pd.to_numeric(df['Movie ID'], errors='coerce')

# Convert the specified columns to integers (removing decimal places)
df['Year'] = df['Year'].astype('Int64')
df['Duration'] = df['Duration'].astype('Int64')
df['Budget'] = df['Budget'].astype('Int64')
df['Revenue'] = df['Revenue'].astype('Int64')
df['Vote Count'] = df['Vote Count'].astype('Int64')
df['Movie ID'] = df['Movie ID'].astype('Int64')

# Convert all object columns to string type (handling NaN as <NA> in string columns)
df['Title'] = df['Title'].astype('string')
df['Director'] = df['Director'].astype('string')
df['Producer'] = df['Producer'].astype('string')
df['Genres'] = df['Genres'].astype('string')
df['Summary'] = df['Summary'].astype('string')
df['Content Rating'] = df['Content Rating'].astype('string')
df['Original Language'] = df['Original Language'].astype('string')
df['Production Companies'] = df['Production Companies'].astype('string')
df['Production Countries'] = df['Production Countries'].astype('string')
df['Spoken Languages'] = df['Spoken Languages'].astype('string')
df['Tagline'] = df['Tagline'].astype('string')

# Drop the 'Adult' column from the DataFrame
df = df.drop(columns=['Adult'])

In [3]:
# Disable scientific notation for floats
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
df.dtypes

Title                   string[python]
Year                             Int64
Director                string[python]
Producer                string[python]
Genres                  string[python]
Summary                 string[python]
Duration                         Int64
Budget                           Int64
Revenue                          Int64
Ratings                        float64
Vote Count                       Int64
Popularity                     float64
Content Rating          string[python]
Original Language       string[python]
Production Companies    string[python]
Production Countries    string[python]
Spoken Languages        string[python]
Tagline                 string[python]
Movie ID                         Int64
dtype: object

In [5]:
# Remove movies that have no summary (NaN or empty strings)
df = df.dropna(subset=['Summary'])

In [6]:
# Get number of rows prior to genre cleaning
pre_genre_clean = len(df)

# Remove films with no genres (NaN or empty)
df = df[df['Genres'].notna() & (df['Genres'] != '')]

# Get number of rows after genre cleaning
post_genre_clean = len(df)

# Check how many movies were removed
removed_no_genre_count = pre_genre_clean - post_genre_clean
print(f"Removed {removed_no_genre_count} films with no genres.")

Removed 1511 films with no genres.


In [7]:
# Replace NaN values with an empty string for the relevant columns
columns_to_process = ['Genres','Original Language', 'Spoken Languages', 'Production Companies', 'Production Countries']
df[columns_to_process] = df[columns_to_process].fillna("")

# Split each of these columns by commas, creating lists from the strings
for col in columns_to_process:
    df[col] = df[col].apply(lambda x: [item.strip() for item in x.split(',')] if x else [])

In [8]:
# Define the list of genres that represent stand-up comedy
standup_genres = ['Comedy', 'Documentary']

# Function to check if a movie is stand-up based on its genres
def is_standup(genres):
    # Sort both lists so that order doesn't affect the comparison
    return sorted(genres) == sorted(standup_genres)

# Get number of rows prior to stand-up removal
pre_genre_clean = len(df)

# Apply the filter to remove stand-up comedy movies
df = df[~df['Genres'].apply(is_standup)]

# Get number of rows after stand-up removal
post_genre_clean = len(df)

# Check how many movies were removed
removed_standup_count = pre_genre_clean - post_genre_clean
print(f"Removed {removed_standup_count} thought as stand-up comedy.")

Removed 136 thought as stand-up comedy.


In [9]:
# Get number of rows prior to stand-up removal
pre_nan_clan = len(df)

# Remove movies with no Director or Producer
df = df.dropna(subset=['Director', 'Producer'])

# Get number of rows after stand-up removal
post_nan_clean = len(df)

# Check how many movies were removed
removed_dir_or_pro = pre_nan_clan - post_nan_clean
print(f"Removed {removed_dir_or_pro} as they had no director or producer.")

Removed 11407 as they had no director or producer.


In [10]:
# Replace NaN values in string columns with empty strings - some movies don't have production companies
string_columns = df.select_dtypes(include=['object', 'string']).columns
df[string_columns] = df[string_columns].fillna("")

In [11]:
# Doesn't appear all string fields were fixed during data aquisition, apply FTFY to all string columns in the DataFrame - some languages won't be fixed
string_columns = df.select_dtypes(include=['string']).columns

# Fix the text in each of the string columns
for col in string_columns:
    df[col] = df[col].apply(ftfy.fix_text)

In [12]:
# Get row count prior to removing duplicate movies
pre_dupe_remove = len(df)

# Remove duplicate movies based on 'Title' and 'Year' - have found many duplicate movies
df = df.drop_duplicates(subset=['Title', 'Year'], keep='first')  # keep='first' retains the first occurrence

# Get row count after removing duplicate movies
post_dupe_remove = len(df)

# Calculate duplicates removed
dupes_removed = pre_dupe_remove - post_dupe_remove

print(dupes_removed)

1299


In [13]:
df

,Title,Year,Director,Producer,Genres,Summary,Duration,Budget,Revenue,Ratings,Vote Count,Popularity,Content Rating,Original Language,Production Companies,Production Countries,Spoken Languages,Tagline,Movie ID
0,Inception,2010,Christopher Nolan,Christopher Nolan,"[Action, Science Fiction, Adventure]","Cobb, a skilled thief who commits corporate es...",148,160000000,825532764,8.37,36315,78.57,14A,[en],"[Legendary Pictures, Syncopy, Warner Bros. Pic...","[United Kingdom, United States of America]","[English, Fran√ßais, Êó•Êú¨Ë™û, Kiswahili]",Your mind is the scene of the crime.,27205
1,Interstellar,2014,Christopher Nolan,Lynda Obst,"[Adventure, Drama, Science Fiction]",The adventures of a group of explorers who mak...,169,165000000,701729206,8.44,35259,172.87,PG,[en],"[Legendary Pictures, Syncopy, Lynda Obst Produ...","[United Kingdom, United States of America]",[English],Mankind was born on Earth. It was never meant ...,157336
2,The Dark Knight,2008,Christopher Nolan,Emma Thomas,"[Drama, Action, Crime, Thriller]",Batman raises the stakes in his war on crime. ...,152,185000000,1004558444,8.52,32600,143.78,14A,[en],"[DC, Legendary Pictures, Syncopy, Isobel Griff...","[United Kingdom, United States of America]","[English, ÊôÆÈÄöËØù]",Welcome to a world without rules.,155
3,Avatar,2009,James Cameron,James Cameron,"[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...",162,237000000,2923706026,7.58,31298,103.52,PG,[en],"[Dune Entertainment, Lightstorm Entertainment,...","[United States of America, United Kingdom]","[English, Espa√±ol]",Enter the world of Pandora.,19995
4,Deadpool,2016,Tim Miller,Ryan Reynolds,"[Action, Adventure, Comedy]",The origin story of former Special Forces oper...,108,58000000,782837347,7.62,30769,143.64,14A,[en],"[20th Century Fox, The Donners' Company, Genre...",[United States of America],[English],Feel the love.,293660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44151,Ray of Hope,2024,Nikki Cole,Ryan Singh,[Documentary],"Rathika Sitsabaiesan, a former Canadian MP, ad...",96,0,0,0.00,0,0.24,14A,[en],[Ryan Singh Productions],[Canada],[],,1310667
44152,All the Weekend's Drama,2024,John Robb Saunders,John Robb Saunders,[Drama],"A wayward, gentle hearted twenty year-old grap...",108,20000,0,0.00,0,0.24,NR,[en],[95.7 Records],[],[],Welcome to the neighborhood. Everyone loves (h...,1309762
44153,The Karamazovs,2024,Anna Brenner,Aisha Jordan,"[Drama, Comedy, Thriller]",Fyodor Karamazov is ill and holed up in his oc...,86,0,0,0.00,0,0.33,NR,[en],[Village Park Productions],[United States of America],[English],"A new film, inspired by an old story",1229318
44162,Graveyard Shark,2024,Matthew A. Peters,Jason A. Covey,[Horror],It is a gripping horror film set in the small ...,90,0,0,0.00,0,0.29,R,[en],[Mad Angel Films],[United States of America],[English],,1265614
